## Title: Wood_Fiber_Concessions

### Description
“Wood fiber concession” refers to an area allocated by a government or other body for establishment of fast-growing tree plantations for the production of timber and wood pulp for paper and paper products.<br>

This data set displays wood fiber concessions as a single layer assembled by aggregating concession data for multiple countries. The data may come from government agencies, NGOs, or other organizations and varies by date and data sources. For more information on concession data for each country please visit the Open Data Portal.<br>

This data was updated in September 2018 to update boundaries for APRIL.<br>

If you are aware of concession data for additional countries, please email us here.

### FLINT
This dataset has been pre-processed/checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: https://datasets.mojaglobal.workers.dev/

### Format
<b>Extent: </b>Indonesia, Republic of Congo, Malaysia<br>
<b>Format</b>: Vector polyon geoJSON<br>
<b>Cordinate system:</b> EPSG:4326 (WGS84)<br>
<b>Year: April 2018</b><br>
<b>Size:</b>  

### Original source
Accessed from Global Forest Watch Portal: https://data.globalforestwatch.org/datasets/eea64218200743a9ab5d5625089d7256_0 31/01/2021. Feature Service: https://services2.arcgis.com/g8WusZB13b9OegfU/arcgis/rest/services/Wood_Fiber_Concessions/FeatureServer/0

### Licence
Dataset has too many sub-components so overarching licence is not possible. Please cite the source and any changes made.

### Citation
Global Forest Watch Portal (Accessed 2020). Wood Fiber Concessions. https://data.globalforestwatch.org/datasets/eea64218200743a9ab5d5625089d7256_0 Accessed 31/01/2021 (note multiple sources of data in this dataset.

Attribution:<br>
Indonesia: Ministry of Forestry, Asia Pulp and Paper, APRIL<br>
Republic of Congo: WRI & Ministry of Agriculture<br>
Sarawak & Sabah, Malaysia: Earthsight Investigations & Global Witness

### Metadata
View https://data.globalforestwatch.org/datasets/eea64218200743a9ab5d5625089d7256_0

### Notes
Limitations of use: This layer is a compilation of concession data from various countries and sources. The quality of these data can vary depending on the source. This layer may not include all existing concessions in a country, and the location of certain concessions can be inaccurate.<br>

Significant overlap, error in geometry (duplicate vertex and non-simple geometry), Indonesian data looks to have multiple resolutions of input data which has led to slither gaps and overlaps. All overlaps are fixed using the code below fixed (which will mean one polygon will win the area over another based on the longest edge shared), and small slithers less than 2ha are removed.

### Processing
See below for arcpy code to fix geometry, gaps and overlaps. It is recommended that additional manual cleaning is undertaken.

In [ ]:
import arcpy
import os

# Input variables
in_folder = r"C:\Users\LennyJenny\Documents\ArcGIS\world\UNFCCC\downloads\WoodFibreConcessions\woodfibreoriginal.gdb"
out_folder = r"C:\Users\LennyJenny\Documents\ArcGIS\world\UNFCCC\downloads\WoodFibreConcessions\json"
fullfield = "source" #this needs to be a field in the original table that is fully populated
smallest = "20000" #smallest area to be fixed in m2 - gaps and slithers
scr = arcpy.CreateFileGDB_management(out_folder, "scratch")
scr_folder = os.path.join(out_folder, "scratch.gdb")

# Environments
workspace = in_folder
arcpy.env.workspace = workspace
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)
arcpy.env.outputZFlag = "Disabled"
arcpy.env.overwriteOutput = True
field = fullfield + " IS NULL or " + fullfield + " = ''"
arcpy.env.parallelProcessingFactor = "100%"

# List features to process
featureclasses = arcpy.ListFeatureClasses()
print(featureclasses)

# Repair/check topology and make FLINT ready
for fc in featureclasses:
    fcname = os.path.join(os.path.splitext(fc)[0])
    outjson = os.path.join(out_folder, fcname)
    whereclause = "FID_" + fcname + " =-1 AND AREA_GEO <= " + smallest +" Or AREA_GEO IS NULL"
    print(fcname + ' processing...')
    
    fLayer = "project_Layer"
    arcpy.management.MakeFeatureLayer(fc, fLayer)
    geomRepair = arcpy.management.RepairGeometry(fLayer, "DELETE_NULL", "OGC")[0]
    
    projectIntersect = os.path.join(scr_folder, "projectIntersect")
    arcpy.analysis.Intersect(fLayer, projectIntersect, "ONLY_FID")
    
    projectSingle = os.path.join(scr_folder, "projectSingle")
    arcpy.management.MultipartToSinglepart(projectIntersect, projectSingle)

    dissolveSlither = os.path.join(scr_folder, "dissolveSlither")
    arcpy.management.Dissolve(projectSingle, dissolveSlither, None, None,"SINGLE_PART")
    
    # Take action if overlaps
    if arcpy.management.GetCount(dissolveSlither)[0] == "0":
        print('no overlaps detected...checking for gaps...')
        
        projectUnion = os.path.join(scr_folder, "projectUnion")
        arcpy.analysis.Union(fLayer,projectUnion, "ALL", None, "NO_GAPS")
        arcpy.management.AddGeometryAttributes(projectUnion, "AREA_GEODESIC", None, "SQUARE_METERS")

        uniSelect = os.path.join(scr_folder, "uniSelect")
        arcpy.analysis.Select(projectUnion, uniSelect, whereclause)
        
        if arcpy.management.GetCount(uniSelect)[0] == "0":
                     
            # Progress report no error
            print(fcname, 'No gaps and overlaps. Repairing geometry and conversion to json...')
    
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fLayer, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fLayer, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")

            print(outjson, '.geojson complete')
            
        else:
            # Take action if gaps
            print('gaps detected')

            appendGap = arcpy.management.Append(uniSelect, fLayer, "NO_TEST")     
            selectGap = arcpy.management.SelectLayerByAttribute(fLayer, "NEW_SELECTION", field)

            fixedlyr = os.path.join(scr_folder, "fixedlyr")
            arcpy.management.Eliminate(selectGap, fixedlyr, "LENGTH")

            # Progress report 
            print(fcname, 'No overlaps but gaps detected and repaired. Repairing geometry and conversion to json...')

            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fixedlyr, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fixedlyr, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")


    else:
        print('Overlaps detected...')
        # Fix overlaps
        projectErase = os.path.join(scr_folder, "projectErase")
        arcpy.analysis.Erase(fLayer, dissolveSlither, projectErase)
        arcpy.management.Append(dissolveSlither, projectErase, "NO_TEST")
    
        selectSlither = arcpy.management.SelectLayerByAttribute(projectErase, "NEW_SELECTION",  field)
        
        eliminateSlither = os.path.join(scr_folder, "eliminateSlither")
        arcpy.management.Eliminate(selectSlither, eliminateSlither, "LENGTH")
        
        print('Overlaps detected and fixed...checking for gaps...')
        
        projectUnion = os.path.join(scr_folder, "projectUnion")
        arcpy.analysis.Union(eliminateSlither, projectUnion, "ALL", None, "NO_GAPS")
        arcpy.management.AddGeometryAttributes(projectUnion, "AREA_GEODESIC", None, "SQUARE_METERS")
        
        uniSelect = os.path.join(scr_folder, "uniSelect")
        whereUnion= "FID_eliminateSlither = -1 AND AREA_GEO <=" + smallest + " OR AREA_GEO IS NULL"
        arcpy.analysis.Select(projectUnion, uniSelect, whereUnion)
        
        if arcpy.management.GetCount(uniSelect)[0] == "0":
            
            # Progress report no error
            print(fcname, ' No gaps detected. Repairing geometry and conversion to json...')
    
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(eliminateSlither, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(eliminateSlither, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")

            print(outjson, '.geojson complete')
            
        else:
            # Take action if gaps
            appendGap = arcpy.management.Append(uniSelect, eliminateSlither, "NO_TEST")
            
            selectGap = arcpy.management.SelectLayerByAttribute(eliminateSlither, "NEW_SELECTION",  field)
            
            fixedlyr = os.path.join(scr_folder, "fixedlyr")
            
            arcpy.management.Eliminate(selectGap, fixedlyr, "LENGTH")
            
            print('gaps detected and repaired')
            
            # Progress report 
            print(fcname, 'Gaps and overlaps fixed. Repairing geometry and conversion to json...')
            
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fixedlyr, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fixedlyr, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
            
            MYS = arcpy.management.SelectLayerByAttribute(fixedlyr, "NEW_SELECTION", "COUNTRY = 'MYS'", None)
            MYSOUTJSON = os.path.join(out_folder, fcname + "_MYS")
            arcpy.conversion.FeaturesToJSON(MYS, MYSOUTJSON, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
            IND = arcpy.management.SelectLayerByAttribute(fixedlyr, "NEW_SELECTION", "COUNTRY = 'IND'", None)
            INDOUTJSON = os.path.join(out_folder, fcname + "_IND")
            arcpy.conversion.FeaturesToJSON(IND, INDOUTJSON, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
            COG = arcpy.management.SelectLayerByAttribute(fixedlyr, "NEW_SELECTION", "COUNTRY = 'COG'", None)
            COGOUTJSON = os.path.join(out_folder, fcname + "_COG")
            arcpy.conversion.FeaturesToJSON(COG, COGOUTJSON, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
arcpy.AddMessage("All done!")
print('done')